# Baseline network

This notebook trains the baseline network with exact same architecture as the one in paper [Recognition of handwritten Latin characters with diacritics using CNN](https://journals.pan.pl/dlibra/publication/136210/edition/119099/content/bulletin-of-the-polish-academy-of-sciences-technical-sciences-recognition-of-handwritten-latin-characters-with-diacritics-using-cnn-lukasik-edyta-charytanowicz-malgorzata-milosz-marek-tokovarov-michail-kaczorowska-monika-czerwinski-dariusz-zientarski-tomasz-2021-69-no-1?language=en)

Model architecture description:

"The architecture of the concrete CNN is shown in Fig. 2.
The input is a 32x32 binarized matrix. The input is then prop-
agated through 12 adaptable layers. First come two convolu-
tional layers having 32 filters with the size of 3x3 and stride 1.
Secondly, the output of the convolutional layer is fed to the
ReLU function. The output is down-sampled using a max-pool-
ing operation with a 2x2 stride. Next, the dropout technique is
used with the coefficient 0.25. The four operations (two con-
volutions, nonlinearity, max-pooling, and dropout) are repeated,
using 64 filters for the convolutional layers. The output of the
last layer is then flattened and fed through a fully connected
layer with 256 neurons and ReLU nonlinearities, dropped out
with the 0.25 coefficient, and a final output layer is fully con-
nected with a Softmax activation function. The Adam optimizer
and the cross-entropy loss function were used in the network. 
The output is a probability distribution over 89 classes."




After training, model is serialized and uploaded to W&B project.

In [2]:
import wandb
import tensorflow as tf
import numpy as np
import pathlib
import shutil
from typing import List

def load_data(run, artifact_name = f"phcd_paper_splits_tfds") -> List[tf.data.Dataset]:
    """
    Downloads datasets from a wandb artifact and loads them into a list of tf.data.Datasets.
    """

    artifact = run.use_artifact(f"master-thesis/{artifact_name}:latest")
    artifact_dir = pathlib.Path(
        f"./artifacts/{artifact.name.replace(':', '-')}"
    ).resolve()
    if not artifact_dir.exists():
        artifact_dir = artifact.download()
        artifact_dir = pathlib.Path(artifact_dir).resolve()

    # if tf.__version__ minor is less than 10, use
    # tf.data.experimental.load instead of tf.data.Dataset.load

    if int(tf.__version__.split(".")[1]) < 10:
        load_function = tf.data.experimental.load
    else:
        load_function = tf.data.Dataset.load
    
    output_list = []
    for split in ["train", "test", "val"]:
        ds = load_function(str(artifact_dir / split), compression="GZIP")
        output_list.append(ds)
    
    return output_list

def get_number_of_classes(ds: tf.data.Dataset) -> int:
    """
    Returns the number of classes in a dataset.
    """
    labels_iterator= ds.map(lambda x, y: y).as_numpy_iterator()
    labels = np.concatenate(list(labels_iterator))
    return len(np.unique(labels))

def preprocess_dataset(ds: tf.data.Dataset, batch_size: int, cache: bool = True) -> tf.data.Dataset:
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))  # normalize
    ds = ds.unbatch().batch(batch_size)
    if cache:
        ds = ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

def calculate_model_size_on_disk(path: str) -> int:
    return pathlib.Path(path).stat().st_size    

def calculate_model_num_parameters(model: tf.keras.Model) -> int:
    return model.count_params()

def calculate_model_flops(summary) -> float:
    # from run.summary get GFLOPs or GFLOPS whichever is available
    print(summary.keys())
    if "GFLOPs" in summary.keys():
        return summary.get("GFLOPs")
    elif "GFLOPS" in summary.keys():
        return summary.get("GFLOPS")

2023-01-24 07:53:51.285714: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Available devices: ", tf.config.list_physical_devices())

Num GPUs Available:  1
Available devices:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2023-01-24 07:53:54.546334: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-24 07:53:54.559553: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-24 07:53:54.559748: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
def train(model_name, defaults, artifact_name="phcd_paper_splits_tfds"):
    with wandb.init(project="master-thesis", job_type="training", name=model_name, config=defaults, tags=[artifact_name]) as run:
        # hyperparameters
        epochs = wandb.config.epochs
        bs = wandb.config.batch_size

        ds_train, ds_test, ds_val = load_data(run, artifact_name=artifact_name)

        num_classes = get_number_of_classes(ds_val)

        ds_train = preprocess_dataset(ds_train, batch_size=bs)
        ds_val = preprocess_dataset(ds_val, batch_size=bs)
        ds_test = preprocess_dataset(ds_test, batch_size=bs, cache=False)

        '''
        The architecture of the concrete CNN is shown in Fig. 2.
        The input is a 32x32 binarized matrix. 
        The input is then propagated through 12 adaptable layers. 
        First come two convolutional layers having 32 filters with the size of 3x3 and stride 1.
        Secondly, the output of the convolutional layer is fed to the
        ReLU function. The output is down-sampled using a max-pool-
        ing operation with a 2x2 stride. Next, the dropout technique is
        used with the coefficient 0.25. The four operations (two con-
        volutions, nonlinearity, max-pooling, and dropout) are repeated,
        using 64 filters for the convolutional layers. The output of the
        last layer is then flattened and fed through a fully connected
        layer with 256 neurons and ReLU nonlinearities, dropped out
        with the 0.25 coefficient, and a final output layer is fully con-
        nected with a Softmax activation function. The Adam optimizer
        and the cross-entropy loss function were used in the network
        [24]. The output is a probability distribution over 89 classes.
        '''

        model = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(32, 32, 1)),
                
                # 2 Convolutional layers with 32 filters, 3x3 size, and stride 1
                tf.keras.layers.Conv2D(32, (3, 3), strides=1, activation='relu'),
                tf.keras.layers.Conv2D(32, (3, 3), strides=1, activation='relu'),
                
                # Max-pooling operation with 2x2 stride
                tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
                # Dropout with coefficient 0.25
                tf.keras.layers.Dropout(0.25),
                
                # Repeat above 4 operations using 64 filters
                tf.keras.layers.Conv2D(64, (3, 3), strides=1, activation='relu'),
                tf.keras.layers.Conv2D(64, (3, 3), strides=1, activation='relu'),
                tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
                tf.keras.layers.Dropout(0.25),
                
                # Flatten the output and feed through fully connected layer with 256 neurons
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(256, activation='relu'),

                # Dropout with coefficient 0.25
                tf.keras.layers.Dropout(0.25),
                tf.keras.layers.Dense(num_classes, activation="softmax"),
            ]
        )

        model.compile(
            optimizer="adam",
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=["accuracy"],
        )

        # save the best model
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=f"./artifacts/{model_name}.h5",
            save_weights_only=False,
            monitor="val_accuracy",
            mode="max",
            save_best_only=True,
        )

        wandb_callback = wandb.keras.WandbCallback(
            save_model=False,
            compute_flops=True,
        )

        history = model.fit(
            ds_train,
            epochs=epochs,
            validation_data=ds_val,
            callbacks=[wandb_callback, checkpoint_callback],
        )

        # calculate model size on disk, flops and number of parameters

        flops = calculate_model_flops(wandb.run.summary)
        disk_size = calculate_model_size_on_disk(f"./artifacts/{model_name}.h5")
        num_parameters = calculate_model_num_parameters(model)

        # evaluate model on ds_test and log to wandb
        test_loss, test_acc = model.evaluate(ds_test)

        wandb.log({
            "test loss": test_loss, 
            "test accuracy": test_acc, 
            "number of parameters": num_parameters,
            "disk size": disk_size, 
            "model flops": flops
            })
            
        # save artifact to wandb
        artifact = wandb.Artifact(
            name=model_name,
            type="model"
        )

        # save best model to artifact
        artifact.add_file(f"./artifacts/{model_name}.h5")
        run.log_artifact(artifact)
        run.finish()

In [5]:
defaults = dict(
    batch_size=32*2,
    epochs=100,    
    optimizer="adam"
)

#train("baseline-uppercase-diacritics", defaults, artifact_name="uppercase_splits_tfds")
train("baseline-numbers", defaults, artifact_name="numbers_splits_tfds")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gratkadlafana. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact numbers_splits_tfds:latest, 50.52MB. 12 files... 
wandb:   12 of 12 files downloaded.  
Done. 0:0:1.0


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-01-24 07:53:59.652209: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 07:53:59.653323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-24 07:53:59.653577: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-24 07:53:59.653794: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_nod

Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Epoch 1/100


2023-01-24 07:54:01.259541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-24 07:54:01.259594: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-24 07:54:01.259785: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-01-24 07:54:01.261267: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-24 07:54:01.261511: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-24 07:54:01.261708: I tensor

675/675 [==============================] - 13s 12ms/step - loss: 0.2237 - accuracy: 0.9292 - val_loss: 0.0485 - val_accuracy: 0.9847
Epoch 2/100
675/675 [==============================] - 7s 11ms/step - loss: 0.0623 - accuracy: 0.9820 - val_loss: 0.0383 - val_accuracy: 0.9885
Epoch 3/100
675/675 [==============================] - 7s 10ms/step - loss: 0.0421 - accuracy: 0.9875 - val_loss: 0.0218 - val_accuracy: 0.9933
Epoch 4/100
675/675 [==============================] - 7s 10ms/step - loss: 0.0368 - accuracy: 0.9889 - val_loss: 0.0158 - val_accuracy: 0.9958
Epoch 5/100
675/675 [==============================] - 7s 10ms/step - loss: 0.0273 - accuracy: 0.9913 - val_loss: 0.0189 - val_accuracy: 0.9935
Epoch 6/100
675/675 [==============================] - 7s 10ms/step - loss: 0.0267 - accuracy: 0.9917 - val_loss: 0.0122 - val_accuracy: 0.9963
Epoch 7/100
675/675 [==============================] - 7s 10ms/step - loss: 0.0204 - accuracy: 0.9938 - val_loss: 0.0096 - val_accuracy: 0.9972
Epo

accuracy,▁▇▇▇████████████████████████████████████
disk size,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model flops,▁
number of parameters,▁
test accuracy,▁
test loss,▁
val_accuracy,▁▅▆▇▇▇██████████████████████████████████
val_loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,0.01429


In [6]:
raise Exception("Stop training")

Exception: Stop training

In [ ]:
train("baseline-uppercase", defaults, artifact_name="uppercase_no_diacritics_splits_tfds")

In [ ]:
train("baseline-lowercase-diacritics", defaults, artifact_name="lowercase_splits_tfds")

In [7]:
train("baseline-lowercase", defaults, artifact_name="lowercase_no_diacritics_splits_tfds")

wandb: Downloading large artifact lowercase_no_diacritics_splits_tfds:latest, 82.10MB. 18 files... 
wandb:   18 of 18 files downloaded.  
Done. 0:0:0.8


Epoch 1/100


2023-01-24 08:10:49.496457: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-24 08:10:49.499152: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2023-01-24 08:10:49.499546: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-01-24 08:10:49.501212: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-24 08:10:49.501962: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-24 08:10:49.502143: I tensor

      4/Unknown - 2s 67ms/step - loss: 3.2627 - accuracy: 0.0508WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0178s vs `on_train_batch_end` time: 0.0240s). Check your callbacks.
1806/1806 [==============================] - 22s 11ms/step - loss: 0.5630 - accuracy: 0.8318 - val_loss: 0.2716 - val_accuracy: 0.9144
Epoch 2/100
1806/1806 [==============================] - 20s 11ms/step - loss: 0.3065 - accuracy: 0.9065 - val_loss: 0.2039 - val_accuracy: 0.9371
Epoch 3/100
1806/1806 [==============================] - 20s 11ms/step - loss: 0.2651 - accuracy: 0.9189 - val_loss: 0.1865 - val_accuracy: 0.9418
Epoch 4/100
1806/1806 [==============================] - 20s 11ms/step - loss: 0.2419 - accuracy: 0.9259 - val_loss: 0.1666 - val_accuracy: 0.9463
Epoch 5/100
1806/1806 [==============================] - 19s 11ms/step - loss: 0.2237 - accuracy: 0.9294 - val_loss: 0.1527 - val_accuracy: 0.9496
Epoch 6/100
1806/1806 [================

accuracy,▁▅▆▆▇▇▇▇▇▇▇▇▇▇▇█████████████████████████
disk size,▁
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
model flops,▁
number of parameters,▁
test accuracy,▁
test loss,▁
val_accuracy,▁▃▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████████
val_loss,█▆▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
GFLOPs,0.01429


In [ ]:
train("baseline", defaults, artifact_name="phcd_paper_splits_tfds")